In [1]:
import torch
import bittensor as bt
import os.path
from multiprocessing import Pool
import pandas as pd
import numpy as np
from substrateinterface import SubstrateInterface, Keypair

subtensor = bt.subtensor('archive')

In [2]:
subtensor.block

4145152

## Global param

In [3]:
substrate = SubstrateInterface(
    url="wss://archive.chain.opentensor.ai:443/",
    ss58_format=42,
    type_registry_preset='legacy'
)

foundation_hk = '5F4tQyWrhfGVcNhoqeiNsR6KjD4wMZ2kfhLj4oHYuyHbZAc3'
corcel_hk = '5GKH9FPPnWSUoeeTJp19wVtd84XqFW4pyK2ijV2GsFbhTrP1'
netuids = range(0, 52)
block = 4144168
metagraph_storage_path = "./metagraphs"
processes = 32

block_hash = substrate.get_block_hash(block)

## Medullar performance 

In [4]:
# download updated metagraphs
metas = {}
for netuid in range(52):
    metas[netuid] = subtensor.metagraph(netuid = netuid, lite = True)
    file_name = f"{metagraph_storage_path}/netuid{netuid}_block{block}.pt"
    torch.save(metas[netuid], file_name)

# load metagraphs
metas = {}
for netuid in range(52):
    file_name = f"{metagraph_storage_path}/netuid{netuid}_block{block}.pt"
    metas[netuid] = torch.load(file_name)
    print(metas[netuid])

/tmp/ipykernel_2221111/302523213.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  metas[netuid] = torch.load(file_name)


metagraph(netuid:0, n:64, block:4145152, network:archive)
metagraph(netuid:1, n:1024, block:4145152, network:archive)
metagraph(netuid:2, n:256, block:4145152, network:archive)
metagraph(netuid:3, n:256, block:4145152, network:archive)
metagraph(netuid:4, n:256, block:4145152, network:archive)
metagraph(netuid:5, n:256, block:4145152, network:archive)
metagraph(netuid:6, n:256, block:4145152, network:archive)
metagraph(netuid:7, n:256, block:4145152, network:archive)
metagraph(netuid:8, n:256, block:4145152, network:archive)
metagraph(netuid:9, n:256, block:4145153, network:archive)
metagraph(netuid:10, n:256, block:4145153, network:archive)
metagraph(netuid:11, n:256, block:4145153, network:archive)
metagraph(netuid:12, n:256, block:4145153, network:archive)
metagraph(netuid:13, n:256, block:4145153, network:archive)
metagraph(netuid:14, n:256, block:4145153, network:archive)
metagraph(netuid:15, n:256, block:4145153, network:archive)
metagraph(netuid:16, n:256, block:4145153, network

In [5]:
tempos = substrate.query_map("SubtensorModule", "Tempo", block_hash=block_hash)

# === Live emission
def get_subnet_emission(block):
    emissions = np.array([])
    for netuid in range(52):
        meta = metas[netuid]
        # print( f"{netuid}, {meta.E.sum():.3f}, {E[netuid].item() * tempos[netuid][1].value * 0.82:.3f}")
        emissions = np.append(emissions, meta.E.sum() / (tempos[netuid][1].value * 0.82))
    return torch.tensor(emissions)

E = get_subnet_emission(None)
E.sum()

tensor(0.8047, dtype=torch.float64)

## Medullar performance compared to corcel

In [6]:
df = []
for netuid in netuids:
    if netuid == 0:
        continue
    meta = metas[netuid]
    
    if foundation_hk in meta.hotkeys:
        uid = meta.hotkeys.index(foundation_hk)
        foundation_vt = meta.validator_trust[uid] 
        foundation_e = meta.E[uid]/ meta.S[uid] / tempos[netuid][1].value  # emission per stake per block
        foundation_daily_e = meta.E[uid] / tempos[netuid][1].value * 7200 # including to norminators
        foundation_uid = uid
    else:
        foundation_vt = None
        foundation_e = None
        foundation_uid = None
        
    if corcel_hk in meta.hotkeys:
        uid = meta.hotkeys.index(corcel_hk)
        corcel_vt = meta.validator_trust[uid] # / meta.S[uid]* 1_000_000 
        corcel_e = meta.E[uid]/ meta.S[uid] / tempos[netuid][1].value  # emission per stake per block
    
    else:    
        corcel_vt = None
        corcel_e = None
    
    df.append({
        "netuid": netuid,
        "emission": E[netuid].item(), 
        "foundation_uid": foundation_uid,
        "foundation_vt": foundation_vt,
        "corcel_vt": corcel_vt,
        "foundation_e": foundation_e,
        "corcel_e": corcel_e,
        "foundation_daily_e": foundation_daily_e,
    })

df = pd.DataFrame(df)
df['under_performing_e'] = df['corcel_e'] - df['foundation_e']  
df['margin'] =  df['under_performing_e'] / df['foundation_e']

synapse_take = 0.08
df['foundation_daily_income_org'] = df.foundation_daily_e * 0.18
df['foundation_daily_income'] = df.foundation_daily_e * (1 + df.margin) * (1 - synapse_take) * 0.18
df['synapse_daily_income'] = df.foundation_daily_e * (1 + df.margin) * synapse_take
df['synapse_take'] = synapse_take
df = df[(df.under_performing_e > 0)].sort_values(by='margin', ascending=False)

In [9]:
df

,netuid,emission,top_vt,foundation_uid,foundation_vt,corcel_vt,foundation_e,corcel_e,foundation_daily_e,under_performing_e,margin,foundation_daily_income_org,foundation_daily_income,synapse_daily_income,synapse_take
36,38,0.003264,0.977218,64.0,0.296910,0.927596,9.943008e-11,3.096335e-10,0.564797,2.102034e-10,2.114083,0.101663,0.291261,0.140706,0.08
24,25,0.025229,0.836698,143.0,0.619440,0.799863,1.523866e-09,2.166244e-09,7.694290,6.423782e-10,0.421545,1.384972,1.811296,0.875022,0.08
43,45,0.010184,0.900877,250.0,0.668208,0.845487,6.026910e-10,8.523032e-10,4.163768,2.496122e-10,0.414163,0.749478,0.975094,0.471060,0.08
2,3,0.003328,0.956634,154.0,0.695857,0.894957,1.887296e-10,2.434271e-10,1.422980,5.469744e-11,0.289819,0.256136,0.303940,0.146831,0.08
40,42,0.004250,0.965454,255.0,0.808347,0.990097,3.001750e-10,3.701812e-10,2.263252,7.000620e-11,0.233218,0.407385,0.462203,0.223287,0.08
44,46,0.000320,0.901503,70.0,0.876387,0.996048,3.326940e-11,3.834188e-11,0.209979,5.072481e-12,0.152467,0.037796,0.040074,0.019360,0.08
28,29,0.040532,0.886625,143.0,0.729274,0.750729,2.580524e-09,2.945526e-09,14.658251,3.650018e-10,0.141445,2.638485,2.770750,1.338527,0.08
20,21,0.014232,0.933501,224.0,0.791287,0.913939,1.023186e-09,1.167895e-09,5.812041,1.447094e-10,0.141430,1.046167,1.098597,0.530723,0.08
7,8,0.024173,0.977722,69.0,0.810956,0.910857,1.525029e-09,1.697393e-09,8.662682,1.723646e-10,0.113024,1.559283,1.596677,0.771342,0.08
27,28,0.015030,0.972869,217.0,0.880903,0.972869,1.036310e-09,1.152873e-09,5.886595,1.165622e-10,0.112478,1.059587,1.084466,0.523897,0.08


## Medullar child graph

In [7]:
substrate = SubstrateInterface(url="wss://entrypoint-finney.opentensor.ai:443")
parent = foundation_hk

results = {}
for netuid in netuids:
    if netuid == 0:
        continue
    try: 
        result = substrate.query('SubtensorModule', 'ChildKeys', [parent, netuid]).value[0]
        results[netuid] = [result[0]/2**64, result[1]]
    except:
        continue

In [8]:
results

{1: [0.1, '5DQ2Geab6G25wiZ4jGH6wJM8fekrm1QhV9hrRuntjBVxxKZm'],
 2: [0.1, '5DQ2Geab6G25wiZ4jGH6wJM8fekrm1QhV9hrRuntjBVxxKZm'],
 3: [0.1, '5DQ2Geab6G25wiZ4jGH6wJM8fekrm1QhV9hrRuntjBVxxKZm'],
 4: [0.1, '5DQ2Geab6G25wiZ4jGH6wJM8fekrm1QhV9hrRuntjBVxxKZm'],
 5: [0.1, '5DQ2Geab6G25wiZ4jGH6wJM8fekrm1QhV9hrRuntjBVxxKZm'],
 6: [0.1, '5DQ2Geab6G25wiZ4jGH6wJM8fekrm1QhV9hrRuntjBVxxKZm'],
 7: [0.1, '5DQ2Geab6G25wiZ4jGH6wJM8fekrm1QhV9hrRuntjBVxxKZm'],
 8: [0.1, '5DQ2Geab6G25wiZ4jGH6wJM8fekrm1QhV9hrRuntjBVxxKZm'],
 9: [0.1, '5DQ2Geab6G25wiZ4jGH6wJM8fekrm1QhV9hrRuntjBVxxKZm'],
 10: [0.1, '5DQ2Geab6G25wiZ4jGH6wJM8fekrm1QhV9hrRuntjBVxxKZm'],
 11: [0.1, '5DQ2Geab6G25wiZ4jGH6wJM8fekrm1QhV9hrRuntjBVxxKZm'],
 12: [0.1, '5DQ2Geab6G25wiZ4jGH6wJM8fekrm1QhV9hrRuntjBVxxKZm'],
 13: [0.1, '5DQ2Geab6G25wiZ4jGH6wJM8fekrm1QhV9hrRuntjBVxxKZm'],
 14: [0.1, '5DQ2Geab6G25wiZ4jGH6wJM8fekrm1QhV9hrRuntjBVxxKZm'],
 15: [0.1, '5DQ2Geab6G25wiZ4jGH6wJM8fekrm1QhV9hrRuntjBVxxKZm'],
 16: [0.1, '5DQ2Geab6G25wiZ4jGH6wJM8fekrm1QhV9hrR